In [ ]:
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
ls

In [ ]:
cd darknet/

In [ ]:
ls

compile the library

In [ ]:
!make

Downlaod YOLO weights

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

In [ ]:
ls

In [ ]:
!./darknet detect cfg/yolov4.cfg yolov4.weights data/person.jpg

In [ ]:
import cv2 
import matplotlib.pyplot as plt
def show_detect(path):
  img=cv2.imread(path)
  plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
  plt.axis('off')
  fig=plt.gcf()
  fig.set_size_inches(18,10) 


In [ ]:
show_detect('predictions.jpg')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
ls

In [ ]:
!./darknet detector demo cfg/coco.data cfg/yolov4.cfg yolov4.weights -dont_show /content/video_street.mp4 -i 0 -out_filename /content/sample_data/video_street_results.avi

In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  Conv2D ,MaxPool2D,Flatten,Dense,BatchNormalization,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import zipfile
path='/content/drive/MyDrive/computer vision/Computer Vision Masterclass/Datasets/fer_images.zip'
zip_o=zipfile.ZipFile(file=path,mode='r')
zip_o.extractall('./')
zip_o.close()

In [ ]:
train_gen=ImageDataGenerator(rescale=1./255,
                                 rotation_range=7,
                                 horizontal_flip=True,
                                 zoom_range=0.2)

In [ ]:
train_data=train_gen.flow_from_directory('/content/fer2013/train',
                                         target_size=(48,48),
                                         batch_size=16,
                                         class_mode="categorical",
                                         shuffle=True)

In [ ]:
np.unique(train_data.classes,return_counts=True)

In [ ]:
train_data.class_indices

In [ ]:
test_gen=ImageDataGenerator(rescale=1./255)
test_data=test_gen.flow_from_directory('/content/fer2013/validation',
                             target_size=(48,48),batch_size=1,
                             class_mode="categorical",
                             shuffle=False)

In [ ]:
detectors=32
width,height=48,48
classes=7
epochs=100

network=Sequential()
network.add(Conv2D(detectors,kernel_size=(3,3),padding='same',activation='relu',input_shape=(width,height,3)))
network.add(BatchNormalization())
network.add(Conv2D(detectors,(3,3),activation='relu',padding='same'))
network.add(BatchNormalization())
network.add(MaxPool2D(pool_size=(2,2)))
network.add(Dropout(0.2))


network.add(Conv2D(2*detectors,(3,3),activation='relu',padding='same'))
network.add(BatchNormalization())
network.add(Conv2D(2*detectors,(3,3),activation='relu',padding='same'))
network.add(Conv2D(2*detectors,(3,3),activation='relu',padding='same'))
network.add(MaxPool2D(pool_size=(2,2)))
network.add(Dropout(0.2))

network.add(Conv2D(2*2*detectors,(3,3),activation='relu',padding='same'))
network.add(BatchNormalization())
network.add(Conv2D(2*2*detectors,(3,3),activation='relu',padding='same'))
network.add(Conv2D(2*2*detectors,(3,3),activation='relu',padding='same'))
network.add(MaxPool2D(pool_size=(2,2)))
network.add(Dropout(0.2))

network.add(Flatten())
network.add(Dense(2*detectors,activation='relu'))
network.add(BatchNormalization())
network.add(Dropout(0.2))


network.add(Dense(classes,activation='softmax'))

network.summary()


In [ ]:
network.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
network.fit(train_data,epochs=5)

In [ ]:
with open('/content/drive/MyDrive/computer vision/Computer Vision Masterclass/Weights/network_emotions.json','r') as json_file:
  json_saved_model=json_file.read()
json_saved_model

In [ ]:
network_loaded=tf.keras.models.model_from_json(json_saved_model)
network_loaded.load_weights('/content/drive/MyDrive/computer vision/Computer Vision Masterclass/Weights/weights_emotions.hdf5')
network_loaded.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=["accuracys"])

In [ ]:
network_loaded.summary()